In [ ]:
class Solver:
    def __init__(self):
        pass

In [ ]:

from __future__ import print_function
from fenics import *
import mshr

%matplotlib inline

totalTime = 5.0            # final time
num_steps = 500    # number of time steps
dt = totalTime / num_steps # time step size
eps = 0.01         # diffusion coefficient

class ChemicalReactionrate:
    
    reactionRate = 10.0           # reaction rate
    
    def __init__(self, atmosphericTemp, cloudCover, solarElevation):
        self.atmosphericTemp = atmosphericTemp
        self.cloudCover = cloudCover
        self.solarElevation = solarElevation
        
        
    def reactionRate(self):
        """Defining the chemical reaction constant """ 
        k1 = 1/60*(0.5699-(9.056*10**3(90-self.solarElevation))**2.546)*(1-0.75*(self.cloudCover/8)**3.4)
        k2 = 1.325*10**6*exp(-1430/self.atmosphericTemp)
        return k2/k1      


#nx = 100
#ny = 100

# Read mesh from file
#mesh = RectangleMesh(Point(0, 0), Point(2.2, 0.41), nx, ny)

grid = mshr.Rectangle(Point(0, 0), Point(2.5, 2.5))
mesh = mshr.generate_mesh(grid, 64) 

# Define function space for velocity
W = VectorFunctionSpace(mesh, 'P', 2)

# Define function space for system of concentrations
P1 = FiniteElement('P', triangle, 1)
element = MixedElement(P1, P1, P1, P1)
V = FunctionSpace(mesh, element)

# Define test functions
v_1, v_2, v_3, v_4 = TestFunctions(V)

# Define functions for velocity and concentrations
w = Function(W)
u = Function(V)
u_n = Function(V)

# Split system functions to access components
u_1, u_2, u_3, u_4 = split(u)
u_n1, u_n2, u_n3, u_n4 = split(u_n)

# Define source terms
f_1 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.1,2)<0.05*0.05 ? 0.1 : 0',
                 degree=1)
f_2 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.3,2)<0.05*0.05 ? 0.1 : 0',
                 degree=1)
f_3 = Constant(0)

f_4 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.5,2)<0.05*0.05 ? 0.1 : 0',
                 degree=1)

# Define expressions used in variational forms
k = Constant(dt)
K = Constant(K)
eps = Constant(eps)



F = ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx \
    + eps*dot(grad(u_1), grad(v_1))*dx + K*u_1*u_2*v_1*dx \
    + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx \
    + eps*dot(grad(u_2), grad(v_2))*dx + K*u_1*u_2*v_2*dx \
    + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx \
    + eps*dot(grad(u_3), grad(v_3))*dx - K*u_1*u_2*v_3*dx + K*u_3*v_3*dx \
    +((u_4 - u_n4) / k)*v_4*dx + dot(w, grad(u_4))*v_4*dx \
    + eps*dot(grad(u_4), grad(v_4))*dx - f_1*v_1*dx - f_2*v_2*dx \
    - f_3*v_3*dx - f_4*v_4*dx
                



# Create time series for reading velocity data
timeseries_w = TimeSeries('Downloads/wind_data21')
#timeseries_w = TimeSeries('navier_stokes_cylinder/velocity_series')
  
    
# Create VTK files for visualization output
vtkfile_u_1 = File('reaction_system/u_1.pvd')
vtkfile_u_2 = File('reaction_system/u_2.pvd')
vtkfile_u_3 = File('reaction_system/u_3.pvd')
vtkfile_u_4 = File('reaction_system/u_4.pvd')


# Create progress bar
progress = Progress('Time-stepping')
#set_log_level(PROGRESS)

# Time-stepping
t = 0
for n in range(num_steps):

    # Update current time
    t += dt

    # Read velocity from file
    timeseries_w.retrieve(w.vector(), t)

    # Solve variational problem for time step
    solve(F == 0, u)

    # Save solution to file (VTK)
    _u_1, _u_2, _u_3, _u_4 = u.split()
    vtkfile_u_1 << (_u_1, t)
    vtkfile_u_2 << (_u_2, t)
    vtkfile_u_3 << (_u_3, t)
    vtkfile_u_4 << (_u_4, t)
    
    # Update previous solution
    u_n.assign(u)

    # Update progress bar
#progress.update(t / T)

# Hold plot
#interactive()

In [ ]:


from fenics import *

class Plume_sim:
    """ 
    Plume transport and reaction in a 2D Domain with homogenous 
    Neumann boundary conditions for u1, u2, u3, and u4; that is
    du_i/dn = 0 for 1 = 1, 2, 3, 4.
    
    ===========         ===================================
    Attributes                       Description
    ===========         ===================================
    cld                 cloud cover
    T                   Atmospheric temperature
    s_elev              solar elevation    
    nx                  number of cells along the x-axis
    ny                  number of cells along the y-axis
           
    """
    
    from math import exp    
    
    def __init__(self, cld, T, s_elev, nx = 100, ny = 100):
        
        """Intialize plume simulation in the domain """
        
        self.cld = cld
        self.T = T
        self.s_elev = s_elev
        self.nx = nx
        self.ny = ny
        
                
    def mesh_grid(self):
        """ Creates a 2D mesh given the number of cells along 
        the x and y co-ordinates, nx and ny """
        return RectangleMesh(Point(0, 0), Point(1, 1), nx, ny)
        
    def velocity_f(self):
        """ Define function space for the velocity """
        W = VectorFunctionSpace(self.mesh_grid, 'P', 2)
        return Function(W)
       
    def conc(self):
        """ Define function space for the system of concentration """
        P1 = FiniteElement('P', triangle, 1)
        element = MixedElement([P1, P1, P1, P1])
        return FunctionSpace(self.mesh_grid, element)
    
    def rx_const(self):
        """Defining the chemical reaction constant """ 
        k1 = 1/60*(0.5699-(9.056*10**3(90-s_elev))**2.546)*(1-0.75*(cld/8)**3.4)
        k2 = 1.325*10**6*exp(-1430/T)
        return k2/k1      
    
    def u(self):
        return Function(self.conc)
        
    def u_n(self):
        return Function(self.conc)
    
    def var_form(self):
        """ Define concentration Functions and test  """
        v_1, v_2, v_3, v_4 = TestFunctions(self.conc)
        
        #Split concentration function to access components
        u_1, u_2, u_3, u_4 = split(self.u)
        u_n1, u_n2, u_n3, u_n4 = split(self.u_n)
        
        #Defining source terms
        f_1 = ""
        f_2 = ""
        f_3 = Constant(0)
        f_4 = ""
                    
        #Defining the variational problem
        return ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx               + eps*dot(grad(u_1), grad(v_1))*dx + self.rx_const*u_1*u_2*v_1*dx                + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx               + eps*dot(grad(u_2), grad(v_2))*dx + self.rx_const*u_1*u_2*v_2*dx                + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx               + eps*dot(grad(u_3), grad(v_3))*dx - self.rx_const*u_1*u_2*v_3*dx + self.rx_const*u_3*v_3*dx               +((u_4 - u_n4) / k)*v_4*dx + dot(w, grad(u_4))*v_4*dx               + eps*dot(grad(u_4), grad(v_4))*dx - f_1*v_1*dx - f_2*v_2*dx               - f_3*v_3*dx - f_4*v_4*dx
                
                
def run_solver():
    
    # Creating an instance of Plume_sim class
    Sim = Plume_sim(10, 283, 45)
    
    # Create time series for reading velocity data
    timeseries_w = TimeSeries('')
       
    # Create VTK files for visualization output
    vtkfile_u_1 = File('')
    vtkfile_u_2 = FIle('')
    vtkfile_u_3 = FIle('')
    vtkfile_u_4 = FIle('')
    
    t = 0
    for n in range(n_steps):
        
        # Update current time
        t += dt      
        
        # Read velocity from file
        timeseries_w.retrieve(w.vector(), t)
        
        # Solve variational problem for the step
        solve(Sim.var_form() ==0, Sim.u())      
        
        # Save solution to file (VTK)
        _u_1, _u_2, _u_3, _u_4 = Sim.u().split()
        vtkfile_u_1 << (_u_1, t)
        vtkfile_u_2 << (_u_2, t)
        vtkfile_u_3 << (_u_3, t)
        vtkfile_u_4 << (_u_4, t)
        
        #update previous solution
        Sim.u_n().assign(Sim.u())    
        
        
if __name__ == '__main__':
    run_solver()  
    


In [55]:
import numpy as np
class ChemicalReactionrate:
    
    reactionRate = 10.0           # reaction rate
    
    def __init__(self, atmosphericTemp, cloudCover, solarElevation):
        self.atmosphericTemp = atmosphericTemp
        self.cloudCover = cloudCover
        self.solarElevation = solarElevation
        
        
    def reactionRate(self, solarElevation, cloudCover, atmosphericTemp):
        """Defining the chemical reaction constant """ 
        k1 = 1/60*(0.5699-(9.056*(10**3)*(90-solarElevation))**2.546)*(1-0.75*(cloudCover/8)**3.4)
        k2 = 1.325*10**6*np.exp(-1430/atmosphericTemp)
        reactionRate = k2/k1      
#         print(reactionRate)
        return reactionRate


#     def __str__(self):
#         return self.reactionRate
    
#     def __repr__(self):
#         return self.reactionRate


In [56]:
ChemicalReactionrate

__main__.ChemicalReactionrate

In [57]:
ChemicalReactionrate.reactionRate

<function __main__.ChemicalReactionrate.reactionRate(self, solarElevation, cloudCover, atmosphericTemp)>

In [58]:
h = ChemicalReactionrate.reactionRate(1, 10, 300)

TypeError: reactionRate() missing 1 required positional argument: 'atmosphericTemp'

In [54]:
dir(h)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_interface__',
 '__array_priority__',
 '__array_struct__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getformat__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setformat__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__'

In [28]:
class calculator:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def addition(x, y):
        added = x + y-3
        print(added)
        
    def subtraction(self, x, y):
        sub = self.x - self.y
        print(sub)

    def multiplication(self, x, y):
        mult = self.x * self.y
        print(mult)

    def division(self, x, y):
        div = self.x / self.y
        print(div)

In [29]:
calculator.addition(4,1)

2


In [ ]:
calculator.multiplication(2,7)

In [ ]:
class calculator:
    
    def addition(x,y):
        added = x + y
        print(added)
        
    def subtraction(x,y):
        sub = x - y
        print(sub)

    def multiplication(x,y):
        mult = x * y
        print(mult)

    def division(x,y):
        div = x / y
        print(div)